In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2
from keras.optimizers import SGD, RMSprop, Adam

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
#path = "ml-20m/ml-20m/"
#path = "ml-20m/ml-10M100K/"
path = "ml-20m/ml-1m/"

In [3]:
ratings = pd.read_csv(path+'ratings.dat', sep='::')
movie_names = pd.read_csv(path+'movies.dat', sep='::')



/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [4]:
usCol = '1'
mvCol = '1193'
rtCol = '5'
users = ratings[usCol].unique()
movies = ratings[mvCol].unique()


userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}


ratings[mvCol] = ratings[mvCol].apply(lambda x: movieid2idx[x])
ratings[usCol] = ratings[usCol].apply(lambda x: userid2idx[x])


user_min, user_max, movie_min, movie_max = (ratings[usCol].min(), 
    ratings[usCol].max(), ratings[mvCol].min(), ratings[mvCol].max())

n_users = ratings[usCol].nunique()
n_movies = ratings[mvCol].nunique()


In [5]:
#latent factors
n_factors = 50


In [6]:
np.random.seed = 42

msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]



Dot product model

In [7]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)


model.compile(Adam(0.001), loss='mse')


In [8]:
model.fit([trn[usCol], trn[mvCol]], trn[rtCol], batch_size=64, nb_epoch=1,
          validation_data=([val[usCol], val[mvCol]], val[rtCol]))


Train on 800555 samples, validate on 199653 samples
Epoch 1/1
221888/800555 [=======>......................] - ETA: 42s - loss: 7.9150

KeyboardInterrupt: 

Use new learning rate

In [9]:
model.optimizer.lr=0.001


In [24]:
model.fit([trn[usCol], trn[mvCol]], trn[rtCol], batch_size=64, nb_epoch=6, 
          validation_data=([val[usCol], val[mvCol]], val[rtCol]))

Train on 800126 samples, validate on 200082 samples
Epoch 1/6
800126/800126 [==============================] - 65s - loss: 3.3718 - val_loss: 1.4356
Epoch 2/6
800126/800126 [==============================] - 66s - loss: 3.3685 - val_loss: 1.4267
Epoch 3/6
800126/800126 [==============================] - 66s - loss: 3.3655 - val_loss: 1.4238
Epoch 4/6
800126/800126 [==============================] - 65s - loss: 3.3644 - val_loss: 1.4198
Epoch 5/6
800126/800126 [==============================] - 66s - loss: 3.3634 - val_loss: 1.4146
Epoch 6/6
800126/800126 [==============================] - 66s - loss: 3.3632 - val_loss: 1.4062


Adding bias to the model

In [10]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')


In [11]:
model.fit([trn[usCol], trn[mvCol]], trn[rtCol], batch_size=64, nb_epoch=6, 
          validation_data=([val[usCol], val[mvCol]], val[rtCol]))

Train on 800555 samples, validate on 199653 samples
Epoch 1/6
800555/800555 [==============================] - 71s - loss: 3.8511 - val_loss: 1.2289
Epoch 2/6
800555/800555 [==============================] - 72s - loss: 1.8720 - val_loss: 1.0618
Epoch 3/6
800555/800555 [==============================] - 71s - loss: 1.1347 - val_loss: 0.8814
Epoch 4/6
800555/800555 [==============================] - 71s - loss: 0.8618 - val_loss: 0.8430
Epoch 5/6
800555/800555 [==============================] - 71s - loss: 0.8356 - val_loss: 0.8334
Epoch 6/6
800555/800555 [==============================] - 70s - loss: 0.8276 - val_loss: 0.8292


State-of-the-art: http://www.mymedialite.net/examples/datasets.html

Neural network model

In [12]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')


In [14]:
nn.fit([trn[usCol], trn[mvCol]], trn[rtCol], batch_size=64, nb_epoch=8,
          validation_data=([val[usCol], val[mvCol]], val[rtCol]))

Train on 800555 samples, validate on 199653 samples
Epoch 1/8
800555/800555 [==============================] - 73s - loss: 1.2948 - val_loss: 0.8545
Epoch 2/8
800555/800555 [==============================] - 74s - loss: 0.9684 - val_loss: 0.8478
Epoch 3/8
800555/800555 [==============================] - 73s - loss: 0.9637 - val_loss: 0.8528
Epoch 4/8
800555/800555 [==============================] - 74s - loss: 0.9605 - val_loss: 0.8472
Epoch 5/8
800555/800555 [==============================] - 74s - loss: 0.9603 - val_loss: 0.8494
Epoch 6/8
800555/800555 [==============================] - 73s - loss: 0.9583 - val_loss: 0.8484
Epoch 7/8
800555/800555 [==============================] - 73s - loss: 0.9587 - val_loss: 0.8470
Epoch 8/8
800555/800555 [==============================] - 73s - loss: 0.9581 - val_loss: 0.8460
